# Textbook exercises
16.3-2 Prove that a binary tree that is not full cannot correspond to an optimal prefix code.
- The depth of a node in a tree will indicate how long its corresponding code is. Since the code for a node is determined by whether it is the left or right child of its ancestors - and the ancestors of their ancestors until the root - it will have the same length as the number of ancestors it has. Thus, an optimal prefix code that minimizes the length of all codes will have a tree of minimum depth. Unless a binary tree is full, it will not be the minimum depth tree of all possible trees of that size. 

16.3-3 What is an optimal Huffman code for the following set of frequencies, based on the first 8 Fibonacci numbers? {a:1 b:1 c:2 d:3 e:5 f:8 g:13 h:21} Can you generalize your answer to find the optimal code when the frequencies are the first n Fibonacci numbers?
- {a:00000001 b:0000001 c:000001 d:00001 e:0001 f:001 g:01 h:1}.
- The longest optimal code for Fib number n will be (n-1)*"0" + "1"

# Huffman encoding

There is python code available at this gist, which will download the collected works of Shakespeare and, when complete, it will compress them using the Huffman compression algorithm.

Implement encode so that it takes a dictionary which maps symbols to probabilities, and then produces the optimal Huffman encoding (i.e. it must return a new dictionary).
Run the code and compress the complete works of Shakespeare. How big is the original file, what is the size of the compressed version? When the compressed version is decompressed is it identical to the original?

Compress the complete works of Shakespeare using other compression methods and see what a poor job they do (eg. gzip, bzip2, zip). Bear in mind that we are using the optimal symbol code with frequencies constructed from the exact corpus that we are looking to compress, so nothing can beat us right?

What is the percentage of 1’s in the compressed version and what is the percentage of 1’s in the uncompressed version?

In [154]:
import heapq
from urllib.request import urlopen
import shutil
import gzip
import os
from collections import defaultdict
from bitarray import bitarray
import pickle
from operator import itemgetter # to make dict sorted list of tuples

# Download the file if need be:
def download_file(url, filename):
    if not os.path.exists(filename):
        response = urlopen(url + filename)
        shutil.copyfileobj(
            gzip.GzipFile(fileobj=response), open(filename, 'wb'))


# build a frequency table:
def build_freq(filename):
    freq = defaultdict(int)
    with open(filename, 'rb') as f:
        for line in f:
            for char in line:
                freq[char] += 1
    total = float(sum(freq.values()))
    return {char: count / total for (char, count) in freq.items()}

class Node():
    def __init__(self,char='internal',freq=0):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None
        self.code = '0'
    def __str__(self):
        return(str(self.char))
    def __lt__(self, other):
        return self.freq < other.freq

    
# insert to encoding tree 
def heap_push(array, key):
    array.append(key)
    array[0], array[-1] = array[-1], array[0]
    min_heapify(array, 0)
    
# Huffman code
def huffman_code(z):
    if z.left != None:
        z.left.code = bitarray(z.code + '0')
        huffman_code(z.left)
    if z.right != None:
        z.right.code = bitarray(z.code + '1')
        huffman_code(z.right)

def inorder(root, dct):
    if root != None:
        inorder(root.left, dct)
        if root.char != "internal":
            dct[root.char] = root.code
        inorder(root.right, dct)    

# Now build the Huffman encoding:
def encode(symb2freq):
    """Huffman encode the given dict mapping symbols to weights.
    Accept a dictionary which maps a symbol to a probability.
    Return a new dictionary which maps a symbol to a bitarray."""
    n = len(symb2freq)
    Q = list([Node(*x) for x in symb2freq.items()]) # 
    heapq.heapify(Q)

    for _ in range(n-1):
        # make an internal node
        z = Node()
        # make the 2 min the children
        z.left = heapq.heappop(Q)
        z.right = heapq.heappop(Q)
        z.freq = z.left.freq + z.right.freq
        heapq.heappush(Q,z)
        
    root = heapq.heappop(Q)
    huffman_code(root)
    symb2code = {}
    inorder(root, symb2code)
    return(symb2code)

# Now compress the file:
def compress(filename, encoding, compressed_name=None):
    if compressed_name is None:
        compressed_name = filename + ".huff"
    output = bitarray()
    with open(filename, 'rb') as f:
        for line in f:
            for char in line:
                output.extend(encoding[char])
    N = len(output)
    with open(compressed_name, 'wb') as f:
        pickle.dump(N, f)
        pickle.dump(encoding, f)
        output.tofile(f)


# Now decompress the file:
def decompress(filename, decompressed_name=None):
    if decompressed_name is None:
        decompressed_name = filename + ".dehuff"
    with open(filename, 'rb') as f:
        N = pickle.load(f)
        encoding = pickle.load(f)
        bits = bitarray()
        bits.fromfile(f)
        bits = bits[:N]

    # Totally cheating here and using a builtin method:
    output = bits.decode(encoding)
    with open(decompressed_name, 'wb') as f:
        f.write(bytes(output))


url = "http://www.gutenberg.org/ebooks/"
filename = "100.txt.utf-8"

download_file(url, filename)
freq = build_freq(filename)
encoding = encode(freq)
compress(filename, encoding)
decompress(filename + ".huff")

# Do you get identical files?